**To get data from the Steam Developer Portal using a Steam API key see our tutorial: Steam_Recommendation_System_777_tp/Environment_setup/Steam_API_Tutorial_for_CS777_Team_Project.md**

In the second part (1.2), we obtain all the game data owned by the user and the last two weeks' game records from the Steam Developer Portal via the user ID file and the Steam API key, and store this information into the corresponding JSON file. The code operations are summarized as follows:

1. **safe_request function**: this function is used to perform HTTP requests and has a retry mechanism to handle possible network errors. It performs a GET request on the given URL and returns a JSON response if the request is successful. It retries if it encounters an HTTP error, especially if it encounters a 502 Bad Gateway error, and waits for some time before retrying.

2. **process_json_obj function **: This function is used to process the JSON response object obtained from the API and format it into the desired object format. Depending on the output file type (user_out_file), it extracts the relevant data from the response and formats it.

3. **dump_user_info function**: this function is the main driver function. It accepts a base URL, a list of user IDs, and an output file path as arguments. It loops through the list of user IDs, constructs the request URL, and calls the safe_request function to get the response. It then processes the response using the process_json_obj function and writes the result to the specified output file.

4. **Main function**: In the main function, the user's API key, the path to the user ID file, and the base URL for the API are specified. then, the user ID file is opened and loaded as JSON data. Then, the dump_user_info function is called using two different base URLs to get the user's owned game information and recently played game information respectively, and the results are written to the corresponding output files.

5. **Output file path**: The output file path is hardcoded in the code, specifying the output path of the user's owned game information and recently played game information.

In summary, this code is mainly used to obtain user game data from the Steam API and store it in a local file for subsequent analysis and processing.

In [5]:
import requests
import json
import time

def safe_request(url, retries=3, delay=2):
    """
    Safely makes a request to the given URL with retries and delay.

    Args:
        url (str): The URL to make the request to.
        retries (int): The number of retries in case of failure.
        delay (int): The delay between retries.

    Returns:
        dict: The JSON response.
    """
    for attempt in range(retries):
        try:
            resp = requests.get(url)
            resp.raise_for_status()
            return resp.json()
        except requests.exceptions.HTTPError as e:
            if resp.status_code == 502:
                print(f"Encountered HTTP 502 Bad Gateway error, retrying {attempt + 1}/{retries} in {delay} seconds...")
                time.sleep(delay)
            else:
                raise e
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
    raise Exception("Reached maximum retry attempts, request failed.")

def process_json_obj(resp, user_out_file, user_id):
    """
    Processes the JSON response object and returns a formatted object.

    Args:
        resp (dict): The JSON response object.
        user_out_file (str): The output file path.
        user_id (str): The user ID.

    Returns:
        dict: The formatted object.
    """
    if 'user_summary' in user_out_file:
        try:
            obj = resp['response']['players'][0]
        except Exception as e:
            obj = {'steamid': user_id}
    elif 'user_owned_games' in user_out_file:
        obj = resp['response']
        obj = {'steamid': user_id, 'game_count': obj.get('game_count', 0), 'games': obj.get('games', [])}
    elif 'user_friend_list' in user_out_file:
        obj = resp['friendslist']
        obj = {'steamid': user_id, 'friends': obj.get('friends', [])}
    elif 'user_recently_played_games' in user_out_file:
        obj = resp['response']
        obj = {'steamid': user_id, 'total_count': obj.get('total_count', 0), 'games': obj.get('games', [])}
    return obj

def dump_user_info(base_url, user_ids, user_out_file):
    """
    Retrieves user information using user IDs and dumps it to a file.

    Args:
        base_url (str): The base URL for API requests.
        user_ids (list): List of user IDs.
        user_out_file (str): The output file path.
    """
    with open(user_out_file, 'w') as f:
        for user_id in user_ids:
            if 'user_friend_list' in user_out_file:
                url_temp = f"{base_url}&steamid={user_id}&relationship=friend"
            elif 'user_owned_games' in user_out_file:
                url_temp = f"{base_url}&steamid={user_id}&include_appinfo=true&include_played_free_games=true"
            elif 'user_recently_played_games' in user_out_file:
                url_temp = f"{base_url}&steamid={user_id}&count=5"
            else:
                url_temp = f"{base_url}&steamid={user_id}"

            resp = safe_request(url_temp)
            if resp:
                try:
                    obj = process_json_obj(resp, user_out_file, user_id)
                    json.dump(obj, f)
                    f.write('\n')
                    print(f"User info for ID {user_id} dumped to file: {user_out_file}")
                except json.JSONDecodeError:
                    print(f"Failed to decode JSON from response. URL: {url_temp}")
            else:
                print(f"No response received for URL: {url_temp}")

    print(f"User info dumped to file: {user_out_file}")

if __name__ == '__main__':
    key = 'ED8D8EE8F67DD482E78FD7CEFC3CC6DE'
    user_ids_file = '/Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_ids.json'

    with open(user_ids_file) as f:
        user_ids = json.load(f)

    base_url_owned_games = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=' + key
    base_url_recently_played_games = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key=' + key

    dump_user_info(base_url_owned_games, user_ids, '/Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json')
    dump_user_info(base_url_recently_played_games, user_ids, '/Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_recently_played_games.json')


User info for ID 76561197960434622 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561197972495328 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561198028573551 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561197970323416 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561198007695232 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561197970565175 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_games.json
User info for ID 76561197968459473 dumped to file: /Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_owned_gam